# Introduction
This project aims to compare the Neighborhoods of two cities that are Startup Capitals of their respective countries. For which we have chosen, Bangalore in India and, The Silicon Valley, San Francisco in United States of America. 
The primary objective of this project is to help Startup offices in Bangalore on deciding a location in San Francisco to open a branch office. A lot of startup look to open office in The Silicone Valley, and we wish to provide them guidance via this project, by helping them find places similar to their current head office, in Bangalore.  


# Data Sources
1.	The neighborhood data of the two cities is taken from Opencity and Wikipedia, respectively. This for which a are being shared bellow :
        1. https://opencity.in/data/bbmp-wards
        1. https://en.wikipedia.org/wiki/List_of_neighborhoods_in_San_Francisco
    The data from open city is in a CSV formal while the data from Wikipedia will be required to be Scrapped before being used. Beautiful Soup library will be used to scrap the data from the Wikipedia page. 
    Bangalore has 199 neighbourhoods, and San Francisco has 114 neighbourhoods.
1.	Geopy library to be used for geocodes. 
1.	Folium library will be utilized to represent the data on maps. 
1.	Scikit-learn will be used to utilize machine learning. 
1.	Foursquare API will be used to gather neighborhood data.


# Importing Data
Importing data is divided into 3 stages. 
   1. The first stage is getting list of neighbourhoods of the four cities from the above Wikipedia links. 
   1. The second stage is getting location of neighbourhoods. 
   1. The third stage is getting the venues in the neighbourhoods from Foursquare.

## Neighbourhood lists of Cities
We already have the links of the Wikipedia pages from which we can get the list of neighbourhoods in each city. Beautiful Soup library is used to extract the information from the wikitables in the pages. This data is stored in a pandas dataframe. Along with the neighbourhoods, the city name, the state name and the country name are stored.

## Geolocation of the Neighbourhoods
The geopy library is used to get the location data of the Neighbourhoods. Now in geopy library, Nominatim service is used. 
For leveraging the free service of Nominatim there is a restriction of 1 call per sec. If the calls are within a second, then a ‘timeout’ error is displayed. Therefore, there needs to be an one sec gap between each call. A sufficient gap of 2 seconds is provided by calling the sleep function.

### Timeout Errors
Even after providing a 2 sec gap in calls, there are timeout errors. So, to handle these errors is simple. Simply call the Nominatim service again for these locations after checking for network connectivity.

### Missing/No Coordinates
Due to spelling errors in the name of some locations, it will not resolve into coordinates. These can be rectified by using different spellings. Some locations will not resolve despite that. Then that data is procured manually searching on Google Maps.


In [1]:
import pandas as pd
import numpy as np
import requests
import time
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from bs4 import BeautifulSoup
!pip install folium
import folium 

#### Importing Bangalore Data

Steps to follow : 

1. Download the Bangalore Neighbourhood data from the Link
1. Then upload the data into the BD2 SQL database in IBM could
1. After the Database has been updated then make a connection with this notebook to the Database
1. Now you can read the data into your SQL file 

_Voila!_

In [2]:
!pip install ipython-sql
%load_ext sql

In [3]:
%sql ibm_db_sa://mdm67023:gmzv%40m0fzls7b1g9@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB

In [4]:
%%sql
select TABNAME,TABSCHEMA from SYSCAT.TABLES where TABSCHEMA='MDM67023'

 * ibm_db_sa://mdm67023:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


tabname,tabschema
PETSALE,MDM67023
INTERNATIONAL_STUDENT_TEST_SCORES,MDM67023
EMPLOYEES,MDM67023
JOB_HISTORY,MDM67023
JOBS,MDM67023
DEPARTMENTS,MDM67023
LOCATIONS,MDM67023
INSTRUCTOR,MDM67023
CHICAGO_SOCIOECONOMIC_DATA,MDM67023
CHICAGO_CRIME_DATA,MDM67023


<p>Here I go practicing HTML here </p>
<p>   These new coders are too enthusiastic :P </p>
<p>
<ul>
<li>We want to use 'BANGALORE_NEIGHBOURHOOD'</li>
<li>I had forgotten the name :P</li>
</ul>
</p>

In [5]:
BN_sql= %sql select * from BANGALORE_NEIGHBOURHOOD
df_bang = BN_sql.DataFrame()
df_bang.head()

 * ibm_db_sa://mdm67023:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


,Ward No,Ward Name,BBMP Zone Name,BBMP Division,BBMP Sub Division,Assembly constituency,MP Constituency
0,1,Kempegowda Ward,Yelahanka,Yelahanka,Yelahanka,Yelahanka,Chikballapur
1,2,Chowdeswari Ward,Yelahanka,Yelahanka,Yelahanka,Yelahanka,Chikballapur
2,3,Atturu,Yelahanka,Yelahanka,Yelahanka,Yelahanka,Chikballapur
3,4,Yelahanka Satellite Town,Yelahanka,Yelahanka,Yelahanka,Yelahanka,Chikballapur
4,5,Jakkuru,Yelahanka,Byatarayanapura,Byatarayanapura,Byatarayanapura,Bangalore North


Importing San Francisco Data

Steps: 

1. Request the webpage
1. Make a Beautiful-Soup Object
1. From the object search for apprioate HTML tag that will provide the data needed
1. Save the data after extracting it from the HTML tags

In [6]:
webPage = requests.get("https://en.wikipedia.org/wiki/List_of_neighborhoods_in_San_Francisco").text
soup = BeautifulSoup(webPage)

In [7]:
soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of neighborhoods in San Francisco - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"ded387a1-1d90-4b27-bf9b-029231b5216f","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_neighborhoods_in_San_Francisco","wgTitle":"List of neighborhoods in San Francisco","wgCurRevisionId":952030286,"wgRevisionId":952030286,"wgArticleId":1487556,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 maint: archived copy as title","Webarchive template archiveis links","Webarchive template 

In [8]:
headlines = soup.find_all(class_='mw-headline')
headlines = headlines[:-4]

In [9]:
sanfran_neighbourhood=[]
for headline in headlines:
    sanfran_neighbourhood.append(headline.text)
print(len(sanfran_neighbourhood))
sanfran_neighbourhood[0:5]

119


['Alamo Square',
 'Anza Vista',
 'Ashbury Heights',
 'Balboa Park',
 'Balboa Terrace']

In [10]:
df_sf = pd.DataFrame(sanfran_neighbourhood)

In [11]:
df_sf.head()

,0
0,Alamo Square
1,Anza Vista
2,Ashbury Heights
3,Balboa Park
4,Balboa Terrace


### Pre-processing Phase

In [12]:
df_sf.columns = ['Neighbourhood']
df_sf['City']='San Francisco'
df_sf['State']='California'
df_sf['Country']='USA'

In [13]:
df_bang = df_bang[['Ward Name']]
df_bang.columns = ['Neighbourhood']
df_bang['City']='Bangalore'
df_bang['State']='Karnataka'
df_bang['Country']='India'

In [14]:
print('The number of Neighbourhoods in banglore are {}'.format(df_bang.shape[0]))
df_bang.head()

The number of Neighbourhoods in banglore are 198


,Neighbourhood,City,State,Country
0,Kempegowda Ward,Bangalore,Karnataka,India
1,Chowdeswari Ward,Bangalore,Karnataka,India
2,Atturu,Bangalore,Karnataka,India
3,Yelahanka Satellite Town,Bangalore,Karnataka,India
4,Jakkuru,Bangalore,Karnataka,India


In [15]:
print('The number of Neighbourhoods in San Francisco are {}'.format(df_sf.shape[0]))
df_sf.head()

The number of Neighbourhoods in San Francisco are 119


,Neighbourhood,City,State,Country
0,Alamo Square,San Francisco,California,USA
1,Anza Vista,San Francisco,California,USA
2,Ashbury Heights,San Francisco,California,USA
3,Balboa Park,San Francisco,California,USA
4,Balboa Terrace,San Francisco,California,USA


In [16]:
df = pd.concat([df_bang,df_sf],ignore_index=True,sort=False)
df

,Neighbourhood,City,State,Country
0,Kempegowda Ward,Bangalore,Karnataka,India
1,Chowdeswari Ward,Bangalore,Karnataka,India
2,Atturu,Bangalore,Karnataka,India
3,Yelahanka Satellite Town,Bangalore,Karnataka,India
4,Jakkuru,Bangalore,Karnataka,India
5,Thanisandra,Bangalore,Karnataka,India
6,Byatarayanapura,Bangalore,Karnataka,India
7,Kodigehalli,Bangalore,Karnataka,India
8,Vidyaranyapura,Bangalore,Karnataka,India
9,Dodda Bommasandra,Bangalore,Karnataka,India


In [17]:
latitude=[]
longitude=[]

In [18]:
df.iterrows()

<generator object DataFrame.iterrows at 0x7f85b9690c50>

In [20]:
# @hidden
hidden_Nominatim='sunnypot1989@gmail.com'

In [21]:
counter = 0
for index, location in df.iterrows():
    counter+=1
    address = location[0] +', '+ location[1] + ', ' + location[2]
    try:
        geolocator = Nominatim(user_agent=hidden_Nominatim)
        loca = geolocator.geocode(address)
    except GeocoderTimedOut as err:
        print(err)
        latitude.append('timeout')
        longitude.append('timeout')
    else:
        print(type(loca))
        if loca is not None:
            latitude.append(loca.latitude)
            longitude.append(loca.longitude)
            print('The geograpical coordinate of {} are {}, {}.'.format(address, loca.latitude, loca.longitude))
        else:
            latitude.append(None)
            longitude.append(None)
            print('None Coordinates for {}'.format(address))
        time.sleep(1.1)

<class 'geopy.location.Location'>
The geograpical coordinate of Kempegowda Ward, Bangalore, Karnataka are 12.997696099999999, 77.53631245866424.
<class 'geopy.location.Location'>
The geograpical coordinate of Chowdeswari Ward, Bangalore, Karnataka are 13.12045925, 77.5796178539993.
<class 'geopy.location.Location'>
The geograpical coordinate of Atturu, Bangalore, Karnataka are 13.1004095, 77.8587249.
<class 'geopy.location.Location'>
The geograpical coordinate of Yelahanka Satellite Town, Bangalore, Karnataka are 13.089139249999999, 77.5827165877116.
<class 'geopy.location.Location'>
The geograpical coordinate of Jakkuru, Bangalore, Karnataka are 13.09831985, 77.62514557763072.
<class 'geopy.location.Location'>
The geograpical coordinate of Thanisandra, Bangalore, Karnataka are 13.054713, 77.6339258.
<class 'geopy.location.Location'>
The geograpical coordinate of Byatarayanapura, Bangalore, Karnataka are 13.062073550000001, 77.59639164365345.
<class 'geopy.location.Location'>
The geogr

<p> If a location has no result from Nominatim, then we will manual search-></p>
        <p> None Coordinates for J P Park, Bangalore, KA </p> 
<p> Earlier we had 317 rows, Now lets check now many Latitute list have None</p>
<p> Earlier we had 317 rows, Now lets check now many Latitute list have given a Time out Error</p>      

In [22]:
print('The number of Location resulting in \'None\' variable is {}'.format(latitude.count(None)))
print('The number of Location resulting in a \'timeout error\'  is {}'.format(latitude.count('timeout')))

The number of Location resulting in 'None' variable is 54
The number of Location resulting in a 'timeout error'  is 0


Now we know that 60 data points are missing, mostly due to spelling errors. To Solve this I have decided to first Edit the spellings and try

In [23]:
df['Latitude'] = latitude
df['Longitude']= longitude
df.head()

,Neighbourhood,City,State,Country,Latitude,Longitude
0,Kempegowda Ward,Bangalore,Karnataka,India,12.997696,77.536312
1,Chowdeswari Ward,Bangalore,Karnataka,India,13.120459,77.579618
2,Atturu,Bangalore,Karnataka,India,13.100409,77.858725
3,Yelahanka Satellite Town,Bangalore,Karnataka,India,13.089139,77.582717
4,Jakkuru,Bangalore,Karnataka,India,13.098320,77.625146


In [24]:
df_missing = df[df['Latitude'].isnull()]

In [25]:
print(df_missing.shape)
df_missing.tail()

(54, 6)


,Neighbourhood,City,State,Country,Latitude,Longitude
283,Polk Gulch,San Francisco,California,USA,NaN,NaN
297,South End,San Francisco,California,USA,NaN,NaN
309,Upper Market,San Francisco,California,USA,NaN,NaN
314,Westwood Highlands,San Francisco,California,USA,NaN,NaN
316,Yerba Buena,San Francisco,California,USA,NaN,NaN


#### Still 54 data points remain empty :

#### Manual Entry from google search

#### hidden cell importing data from My Projects

In [26]:
# The code was removed by Watson Studio for sharing.

,Unnamed: 0,Neighbourhood,City,State,Country,Latitude,Longitude
0,16,J P Park,Bangalore,Karnataka,India,12.0339,77.5500
1,32,Manorayana Palya,Bangalore,Karnataka,India,13.0302,77.6041
2,43,Marappana Palya,Bangalore,Karnataka,India,13.0206,77.5516
3,57,New Tippa Sandra,Bangalore,Karnataka,India,12.9718,77.6552
4,59,Sagayara Puram,Bangalore,Karnataka,India,13.0071,77.6171


In [27]:
missing_geocodes.set_index('Unnamed: 0',inplace=True)
missing_geocodes.index.name = 'Index'
missing_geocodes.head()

,Neighbourhood,City,State,Country,Latitude,Longitude
Index,,,,,,
16,J P Park,Bangalore,Karnataka,India,12.0339,77.5500
32,Manorayana Palya,Bangalore,Karnataka,India,13.0302,77.6041
43,Marappana Palya,Bangalore,Karnataka,India,13.0206,77.5516
57,New Tippa Sandra,Bangalore,Karnataka,India,12.9718,77.6552
59,Sagayara Puram,Bangalore,Karnataka,India,13.0071,77.6171


In [29]:
df2=df.copy()
for i,data in missing_geocodes.iterrows():
    df.set_value(i,['Latitude'],data['Latitude'])
    df.set_value(i,['Longitude'],data['Longitude'])

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [30]:
import collections
collections.Counter(df['Latitude'].isnull().values)

Counter({False: 314, True: 3})

In [31]:
df[df['Latitude'].isnull()==True]

,Neighbourhood,City,State,Country,Latitude,Longitude
200,Ashbury Heights,San Francisco,California,USA,NaN,NaN
253,Little Hollywood,San Francisco,California,USA,NaN,NaN
297,South End,San Francisco,California,USA,NaN,NaN


In [32]:
df.drop(index=[200,253,297],inplace=True)

checking if there are any more empty datapoints left 

In [33]:
collections.Counter(df['Latitude'].isnull().values)

Counter({False: 314})

In [34]:
#Testing one of the data point that was empty earlier : 
print(df.loc[59]['Latitude'] , df.loc[59]['Longitude'])

13.0071 77.6171


**_Voila!_**

<p><i>Hence, We have no null values, so I am going to save the data and continue from here.</i></p>

In [35]:
# The code was removed by Watson Studio for sharing.

In [36]:
project.save_data(file_name = "DF_LL.csv",data = df.to_csv(index=False),overwrite=True)

{'file_name': 'DF_LL.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'capstoneproject-donotdelete-pr-p77xaxowhj9d4v',
 'asset_id': 'fd4f06a0-10e1-415b-9a7a-724840afc203'}

In [37]:
address = 'San Francisco, California'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Francisco are {}, {}.'.format(latitude, longitude))


map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

markers_colors = []
for lat, lon, poi in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    label = folium.Popup(str(poi))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='rd',
        fill_opacity=0.3).add_to(map_clusters)

map_clusters


The geograpical coordinate of San Francisco are 37.7790262, -122.4199061.


### NOW WE CAN CONTINUE FROM HERE WITHOUT RE-RUNNING THE UPPER PART OF THE PROJECT....PHEW

In [38]:
# The code was removed by Watson Studio for sharing.

In [ ]:
# For future refrence, This is not working and see the next cell 
df=pd.DataFrame(project.get_file("DF_LL.csv"))

In [39]:
# The code was removed by Watson Studio for sharing.

,Neighbourhood,City,State,Country,Latitude,Longitude
0,Kempegowda Ward,Bangalore,Karnataka,India,12.997696,77.536312
1,Chowdeswari Ward,Bangalore,Karnataka,India,13.120459,77.579618
2,Atturu,Bangalore,Karnataka,India,13.100409,77.858725
3,Yelahanka Satellite Town,Bangalore,Karnataka,India,13.089139,77.582717
4,Jakkuru,Bangalore,Karnataka,India,13.098320,77.625146


#### Connecting to foursquare

In [48]:
from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

import json

from pandas.io.json import json_normalize

In [49]:
df_blr = df[df['State']=='Karnataka']
df_blr.shape

(198, 6)

In [50]:
df_sf = df[df['State']=='California']
df_sf.shape

(116, 6)

In [51]:
# The code was removed by Watson Studio for sharing.

In [52]:
# The code was removed by Watson Studio for sharing.

In [53]:
def getNearbyVenues(names, latitudes, longitudes, state, radius=500):
    
    venues_list=[]
    for name, lat, lng, state in zip(names, latitudes, longitudes, state):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            state,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood',
                    'State',
                  'N\'hood Latitude', 
                  'N\'hood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
     
#     print(nearby_venues.head())
    return(nearby_venues)

In [54]:
blr_venues = getNearbyVenues(names=df_blr['Neighbourhood'],
                                   latitudes=df_blr['Latitude'],
                                   longitudes=df_blr['Longitude'],
                                    state=df_blr['State']
                                  )

In [55]:
print(blr_venues.shape)
blr_venues.head()

(1389, 8)


,Neighbourhood,State,N'hood Latitude,N'hood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Kempegowda Ward,Karnataka,12.997696,77.536312,Cafe Coffee Day,12.997306,77.540433,Café
1,Kempegowda Ward,Karnataka,12.997696,77.536312,Liquor mart,12.996410,77.540195,Liquor Store
2,Kempegowda Ward,Karnataka,12.997696,77.536312,Pizza Hut,12.997148,77.540446,Pizza Place
3,Kempegowda Ward,Karnataka,12.997696,77.536312,Sambhav moorthy park,12.997491,77.537856,Park
4,Kempegowda Ward,Karnataka,12.997696,77.536312,Canara Stores,12.995387,77.537896,Department Store


In [57]:
sf_venues = getNearbyVenues(names=df_sf['Neighbourhood'],
                                   latitudes=df_sf['Latitude'],
                                   longitudes=df_sf['Longitude'],
                                    state=df_sf['State']
                                  )

In [58]:
print(sf_venues.shape)
sf_venues.head()

(5056, 8)


,Neighbourhood,State,N'hood Latitude,N'hood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alamo Square,California,37.77636,-122.434689,Alamo Square,37.776045,-122.434363,Park
1,Alamo Square,California,37.77636,-122.434689,Alamo Square Dog Park,37.775878,-122.435740,Dog Run
2,Alamo Square,California,37.77636,-122.434689,Painted Ladies,37.776120,-122.433389,Historic Site
3,Alamo Square,California,37.77636,-122.434689,The Independent,37.775573,-122.437835,Rock Club
4,Alamo Square,California,37.77636,-122.434689,The Mill,37.776425,-122.437970,Bakery


In [59]:
df_blr.head()

,Neighbourhood,City,State,Country,Latitude,Longitude
0,Kempegowda Ward,Bangalore,Karnataka,India,12.997696,77.536312
1,Chowdeswari Ward,Bangalore,Karnataka,India,13.120459,77.579618
2,Atturu,Bangalore,Karnataka,India,13.100409,77.858725
3,Yelahanka Satellite Town,Bangalore,Karnataka,India,13.089139,77.582717
4,Jakkuru,Bangalore,Karnataka,India,13.098320,77.625146


In [60]:
blr_grouped = blr_venues.groupby(['Neighbourhood']).count()

In [61]:
print('Missing Data from {} Locations'.format(df_blr.shape[0]-blr_grouped.shape[0]))

Missing Data from 31 Locations


In [62]:
missing_nhood=[]
for i in df_blr['Neighbourhood']:
    if i not in blr_grouped.index:
        missing_nhood.append(i)

In [63]:
df_blr2 = df_blr.set_index(['Neighbourhood'],drop=True)

In [64]:
for location in missing_nhood:
    
    latitude = df_blr2.loc[location]['Latitude']
    longitude = df_blr2.loc[location]['Longitude']
    print("Latitude = {} & Longitude = {} of Neighbourhood - {} ".format(latitude,longitude,location))

    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            radius, 
            LIMIT)

            # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    print(results)

Latitude = 13.09831985 & Longitude = 77.62514557763072 of Neighbourhood - Jakkuru 
[]
Latitude = 13.3352382 & Longitude = 77.6378555 of Neighbourhood - Kodigehalli 
[]
Latitude = 13.05633975 & Longitude = 77.55711843496857 of Neighbourhood - Dodda Bommasandra 
[]
Latitude = 12.7528354 & Longitude = 77.7305926 of Neighbourhood - Shettihalli 
[]
Latitude = 13.008148599999998 & Longitude = 77.7837101 of Neighbourhood - Mallasandra 
[]
Latitude = 12.0339 & Longitude = 77.55 of Neighbourhood - J P Park 
[]
Latitude = 13.0187848 & Longitude = 77.61511440000001 of Neighbourhood - Kushal Nagar 
[]
Latitude = 12.983417300000001 & Longitude = 77.3918735 of Neighbourhood - Gangenahalli 
[]
Latitude = 12.8831037 & Longitude = 77.1827712 of Neighbourhood - Mattikere 
[]
Latitude = 13.187970800000002 & Longitude = 77.9010899 of Neighbourhood - Chokkasandra 
[]
Latitude = 13.0180855 & Longitude = 77.48593516675216 of Neighbourhood - Dodda Bidarakallu 
[]
Latitude = 13.01582965 & Longitude = 77.603111

_We have recieved null data from Foursquare for the above location. We will proceed for the project purposes, and get back getting moreaccurate venue data when the project submission is over_ 

_We will observe a smiliar trent in San Francisco_

In [130]:
pd.DataFrame(missing_nhood)

,0
0,Jakkuru
1,Kodigehalli
2,Dodda Bommasandra
3,Shettihalli
4,Mallasandra
5,J P Park
6,Kushal Nagar
7,Gangenahalli
8,Mattikere
9,Chokkasandra


In [65]:
df_venues = pd.concat([blr_venues,sf_venues])

In [66]:
print(df_venues.shape)
df_venues.head()

(6445, 8)


,Neighbourhood,State,N'hood Latitude,N'hood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Kempegowda Ward,Karnataka,12.997696,77.536312,Cafe Coffee Day,12.997306,77.540433,Café
1,Kempegowda Ward,Karnataka,12.997696,77.536312,Liquor mart,12.996410,77.540195,Liquor Store
2,Kempegowda Ward,Karnataka,12.997696,77.536312,Pizza Hut,12.997148,77.540446,Pizza Place
3,Kempegowda Ward,Karnataka,12.997696,77.536312,Sambhav moorthy park,12.997491,77.537856,Park
4,Kempegowda Ward,Karnataka,12.997696,77.536312,Canara Stores,12.995387,77.537896,Department Store


### Functions for One-Hot Encoding & the like 
PS: based on Venue Category

In [67]:
def onehot_df(df_venues):
    df_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")
    df_onehot['Neighbourhood'] = df_venues['Neighbourhood'] 
    # move neighborhood column to the first column
    fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
    df_onehot = df_onehot[fixed_columns]

    return df_onehot

In [68]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [69]:
def sorted_dataframe(df_grouped):
    num_top_venues = 20

    indicators = ['st', 'nd', 'rd']

    # create columns according to number of top venues
    columns = ['Neighbourhood']
    for ind in np.arange(num_top_venues):
        try:
            columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
        except:
            columns.append('{}th Most Common Venue'.format(ind+1))

    # create a new dataframe
    df_venues_sorted = pd.DataFrame(columns=columns)
    df_venues_sorted['Neighbourhood'] = df_grouped['Neighbourhood']

    for ind in np.arange(df_grouped.shape[0]):
        df_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], num_top_venues)

    return df_venues_sorted

In [70]:
def map_city_clusters(df_merged,latitude,longitude):
    # create map
    map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
    
    rainbow = ['#ff0000', '#8000ff', '#3c68f9', '#d4dd80', '#f6be68', '#ff964f', '#2adddd', '#b2f396']
    indices = df_merged.groupby('Cluster Labels').count().sort_values(by='City', ascending=False).index.values
    # set color scheme for the clusters
    x = np.arange(kclusters)
    ys = [i+x+(i*x)**2 for i in range(kclusters)]
    #colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    #rainbow = [colors.rgb2hex(i) for i in colors_array]
    

    # add markers to the map
    markers_colors = []
    for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighbourhood'], df_merged['Cluster Labels']):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        indice = np.where(indices == (cluster))[0][0]
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[indice],
            fill=True,
            fill_color=rainbow[indice],
            fill_opacity=0.7).add_to(map_clusters)

    return map_clusters

#### BANGLORE CLUSTERING

In [71]:
blr_onehot = onehot_df(blr_venues)
print(blr_onehot.shape)
blr_onehot.head()

(1389, 186)


,Neighbourhood,ATM,Accessories Store,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Tibetan Restaurant,Toll Plaza,Track Stadium,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Kempegowda Ward,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Kempegowda Ward,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Kempegowda Ward,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Kempegowda Ward,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Kempegowda Ward,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [72]:
blr_grouped = blr_onehot.groupby('Neighbourhood').mean().reset_index()
print(blr_grouped.shape)
blr_grouped.head()

(167, 186)


,Neighbourhood,ATM,Accessories Store,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Tibetan Restaurant,Toll Plaza,Track Stadium,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,A Narayanapura,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,Adugodi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,Agaram,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,Agrahara Dasarahalli,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0
4,Anjanapura,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


##### why do I do this ?
blr_onehot_test = blr_onehot.groupby('Neighbourhood').sum().reset_index()
blr_onehot_test.head()

In [73]:
blr_venues_sorted = sorted_dataframe(blr_grouped)
blr_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,A Narayanapura,Pharmacy,Pizza Place,Fast Food Restaurant,Dessert Shop,Bus Stop,Women's Store,Eastern European Restaurant,Financial or Legal Service,Film Studio,...,Farm,Event Space,Electronics Store,Donut Shop,Dumpling Restaurant,Cosmetics Shop,Dog Run,Diner,Dim Sum Restaurant,Dhaba
1,Adugodi,Ice Cream Shop,Bus Station,Playground,Athletics & Sports,Café,Women's Store,Electronics Store,Fish & Chips Shop,Financial or Legal Service,...,Fast Food Restaurant,Farmers Market,Farm,Event Space,Dumpling Restaurant,Eastern European Restaurant,Flea Market,Donut Shop,Dog Run,Diner
2,Agaram,Burger Joint,Bus Station,Women's Store,Electronics Store,Fish & Chips Shop,Financial or Legal Service,Film Studio,Fast Food Restaurant,Farmers Market,...,Event Space,Eastern European Restaurant,Flea Market,Dumpling Restaurant,Donut Shop,Dog Run,Diner,Dim Sum Restaurant,Dhaba,Dessert Shop
3,Agrahara Dasarahalli,Indian Restaurant,Vegetarian / Vegan Restaurant,Bus Station,Café,Bar,Bakery,Breakfast Spot,Bagel Shop,Women's Store,...,Fish Market,Fish & Chips Shop,Financial or Legal Service,Film Studio,Fast Food Restaurant,Event Space,Farm,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
4,Anjanapura,Pool,Women's Store,Dumpling Restaurant,Fish & Chips Shop,Financial or Legal Service,Film Studio,Fast Food Restaurant,Farmers Market,Farm,...,Electronics Store,Eastern European Restaurant,Donut Shop,Flea Market,Dog Run,Diner,Dim Sum Restaurant,Dhaba,Dessert Shop,Department Store


In [74]:
#set number of clusters
kclusters = 5

blr_grouped_clustering = blr_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans1 = KMeans(n_clusters=kclusters, random_state=0).fit(blr_grouped_clustering)

# check cluster labels generated for each row in the dataframe
len(kmeans1.labels_)

167

**Note** : _df_blr is not sorted, while df_grouped was sorted alphabetically on Neighbourhood_

In [75]:
df_blr_labels = df_blr.sort_values(['Neighbourhood'])
x=[]
for i in df_blr_labels['Neighbourhood']:
    if i in blr_grouped['Neighbourhood'].values:
        x.append(True)
    else : x.append(False)
df_blr_labels=df_blr_labels[x]
print(df_blr_labels.shape)

(167, 6)


In [76]:
df_blr_labels['Cluster Labels'] = kmeans1.labels_
df_blr_labels.head() 

,Neighbourhood,City,State,Country,Latitude,Longitude,Cluster Labels
55,A Narayanapura,Bangalore,Karnataka,India,12.993194,77.670377,3
146,Adugodi,Bangalore,Karnataka,India,12.942847,77.610416,0
113,Agaram,Bangalore,Karnataka,India,12.945704,77.637886,3
104,Agrahara Dasarahalli,Bangalore,Karnataka,India,12.979063,77.540682,4
195,Anjanapura,Bangalore,Karnataka,India,12.858081,77.558071,3


In [77]:
address = 'Bengaluru, India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore are 12.9791198, 77.5912997.


In [78]:
map_city_clusters(df_blr_labels,latitude,longitude)

In [79]:
df_blr_labels.loc[df_blr_labels['Cluster Labels']==0,df_blr_labels.columns[[0] + list(range(6, df_blr_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
146,Adugodi,0
179,Banashankari Temple Ward,0
157,Deepanjali Nagar,0
23,HBR Layout,0
89,Halsoor,0


In [80]:
df_blr_labels.loc[df_blr_labels['Cluster Labels']==1,df_blr_labels.columns[[0] + list(range(6, df_blr_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
72,Kottegepalya,1
124,Marenahalli,1
47,Muneshwara Nagar,1
134,Padarayanapura,1
69,Rajagopal Nagar,1


In [81]:
df_blr_labels.loc[df_blr_labels['Cluster Labels']==2,df_blr_labels.columns[[0] + list(range(6, df_blr_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
133,Bapuji Nagar,2
174,Bommanahalli,2
6,Byatarayanapura,2
1,Chowdeswari Ward,2
156,Gali Anjenaya Temple Ward,2


In [82]:
df_blr_labels.loc[df_blr_labels['Cluster Labels']==3,df_blr_labels.columns[[0] + list(range(6, df_blr_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
55,A Narayanapura,3
113,Agaram,3
195,Anjanapura,3
192,Arakere,3
140,Azad Nagar,3


In [83]:
df_blr_labels.loc[df_blr_labels['Cluster Labels']==4,df_blr_labels.columns[[0] + list(range(6, df_blr_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
104,Agrahara Dasarahalli,4
34,Aramane Nagara,4
2,Atturu,4
175,BTM Layout,4
26,Banasavadi,4


In [139]:
df_blr_labels[['Cluster Labels','Neighbourhood']].groupby("Cluster Labels").count()

,Neighbourhood
Cluster Labels,
0,16
1,7
2,17
3,58
4,69


#### SAN FRANCISCO CLUSTERING

In [84]:
sf_onehot = onehot_df(sf_venues)
print(sf_onehot.shape)
sf_onehot.head()

(5056, 352)


,Neighbourhood,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Alternative Healer,American Restaurant,Animal Shelter,Antique Shop,...,Vineyard,Warehouse,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Alamo Square,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Alamo Square,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Alamo Square,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Alamo Square,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Alamo Square,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
sf_grouped=sf_onehot.groupby('Neighbourhood').mean().reset_index()
print(sf_grouped.shape)
sf_grouped.head()

(116, 352)


,Neighbourhood,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Alternative Healer,American Restaurant,Animal Shelter,Antique Shop,...,Vineyard,Warehouse,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Alamo Square,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.013514,...,0.0,0.0,0.0,0.0,0.027027,0.0,0.0,0.0,0.0,0.013514
1,Anza Vista,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
2,Balboa Park,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.023810,0.0,0.0,0.0,0.0,0.000000
3,Balboa Terrace,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
4,Bayview,0.0,0.0,0.0,0.0,0.0,0.0,0.055556,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000


In [86]:
sf_venues_sorted = sorted_dataframe(sf_grouped)
sf_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Alamo Square,Bar,Ethiopian Restaurant,BBQ Joint,Hotel,Pizza Place,Liquor Store,Sushi Restaurant,Café,Seafood Restaurant,...,Park,Record Shop,Comic Shop,Diner,Pet Store,Pakistani Restaurant,Dog Run,Yoga Studio,Coffee Shop,Cocktail Bar
1,Anza Vista,Café,Sandwich Place,Cosmetics Shop,Big Box Store,Tunnel,Arts & Crafts Store,Grocery Store,Donut Shop,Coffee Shop,...,Bus Line,Health & Beauty Service,Juice Bar,Mexican Restaurant,Southern / Soul Food Restaurant,Burger Joint,Neighborhood,Food,Food & Drink Shop,Flower Shop
2,Balboa Park,Breakfast Spot,Bus Station,Park,Convenience Store,Coffee Shop,Trail,Scenic Lookout,Organic Grocery,Cheese Shop,...,Mexican Restaurant,Bar,Middle Eastern Restaurant,Gift Shop,Bakery,Road,Toy / Game Store,Hardware Store,Restaurant,Café
3,Balboa Terrace,Eye Doctor,Dentist's Office,Park,Fried Chicken Joint,Scenic Lookout,Football Stadium,Food & Drink Shop,Farmers Market,Fast Food Restaurant,...,Flea Market,Flower Shop,Food,Food Stand,Food Court,Falafel Restaurant,Food Truck,Fountain,French Restaurant,Frozen Yogurt Shop
4,Bayview,Southern / Soul Food Restaurant,Bakery,Dance Studio,Light Rail Station,Chinese Restaurant,Mexican Restaurant,Gym,Café,BBQ Joint,...,Pharmacy,Home Service,Dumpling Restaurant,American Restaurant,Piercing Parlor,Pool,Food & Drink Shop,Filipino Restaurant,Fried Chicken Joint,French Restaurant


In [92]:
#set number of clusters
kclusters = 5

sf_grouped_clustering = sf_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans1 = KMeans(n_clusters=kclusters, random_state=0).fit(sf_grouped_clustering)

# check cluster labels generated for each row in the dataframe
len(kmeans1.labels_)

116

**Note** : _df_sf is not sorted, while df_grouped was sorted alphabetically on Neighbourhood_

In [93]:
df_sf.shape

(116, 6)

In [94]:
df_sf_labels = df_sf.sort_values(['Neighbourhood'])
x=[]
for i in df_sf_labels['Neighbourhood']:
    if i in sf_grouped['Neighbourhood'].values:
        x.append(True)
    else : x.append(False)
df_sf_labels=df_sf_labels[x]
print(df_sf_labels.shape)

(116, 6)


In [95]:
df_sf_labels['Cluster Labels'] = kmeans1.labels_
df_sf_labels.head() 

,Neighbourhood,City,State,Country,Latitude,Longitude,Cluster Labels
198,Alamo Square,San Francisco,California,USA,37.776360,-122.434689,4
199,Anza Vista,San Francisco,California,USA,37.780836,-122.443149,4
200,Balboa Park,San Francisco,California,USA,37.765300,-122.445300,4
201,Balboa Terrace,San Francisco,California,USA,32.809471,-117.208557,0
202,Bayview,San Francisco,California,USA,37.728889,-122.392500,4


In [96]:
address = 'San Francisco, California'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Francisco are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of San Francisco are 37.7790262, -122.4199061.


In [97]:
map_city_clusters(df_sf_labels,latitude,longitude)

In [98]:
df_sf_labels.loc[df_sf_labels['Cluster Labels']==0,df_sf_labels.columns[[0] + list(range(6, df_sf_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
201,Balboa Terrace,0
213,Clarendon Heights,0
230,Forest Hill,0
233,Golden Gate Heights,0
237,India Basin,0


In [99]:
df_sf_labels.loc[df_sf_labels['Cluster Labels']==1,df_sf_labels.columns[[0] + list(range(6, df_sf_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
249,Lakeshore,1
251,Lincoln Manor,1
291,Sea Cliff,1
292,Sherwood Forest,1
305,University Mound,1


In [100]:
df_sf_labels.loc[df_sf_labels['Cluster Labels']==2,df_sf_labels.columns[[0] + list(range(6, df_sf_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
217,Crocker-Amazon,2
252,Little Russia,2


In [101]:
df_sf_labels.loc[df_sf_labels['Cluster Labels']==3,df_sf_labels.columns[[0] + list(range(6, df_sf_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
231,Forest Knolls,3


In [102]:
df_sf_labels.loc[df_sf_labels['Cluster Labels']==4,df_sf_labels.columns[[0] + list(range(6, df_sf_labels.shape[1]))]].head()

,Neighbourhood,Cluster Labels
198,Alamo Square,4
199,Anza Vista,4
200,Balboa Park,4
202,Bayview,4
203,Belden Place,4


In [140]:
df_sf_labels[['Cluster Labels','Neighbourhood']].groupby("Cluster Labels").count()

,Neighbourhood
Cluster Labels,
0,18
1,5
2,2
3,1
4,90


### VISUALIZING THE TWO CITIES ON THE WORLD MAP

In [103]:
city_loca=[]
for city in ['Bengaluru, India','San Francisco, California']:
    address = city
    geolocator = Nominatim(user_agent='sunnypot1989@gmail.com')
    loca = geolocator.geocode(address)
    city_loca.append([city, loca.longitude, loca.latitude])
    time.sleep(2.0)
city_loca

[['Bengaluru, India', 77.5912997, 12.9791198],
 ['San Francisco, California', -122.4199061, 37.7790262]]

In [104]:
#add markers to the cities
map = folium.Map()
for i in city_loca:
    label,lng,lat = i[0],i[1],i[2]
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map)  
map

### CLUSTERING BANGLORE AND SAN FRANCISCO TOGETHER

In [105]:
df_onehot = onehot_df(df_venues)
print(df_onehot.shape)
df_onehot.head()

(6445, 399)


,Neighbourhood,ATM,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Alternative Healer,American Restaurant,Andhra Restaurant,...,Vineyard,Warehouse,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Kempegowda Ward,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Kempegowda Ward,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Kempegowda Ward,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Kempegowda Ward,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Kempegowda Ward,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [106]:
df_grouped=df_onehot.groupby('Neighbourhood').mean().reset_index()
print(df_grouped.shape)
df_grouped.head()

(283, 399)


,Neighbourhood,ATM,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Alternative Healer,American Restaurant,Andhra Restaurant,...,Vineyard,Warehouse,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,A Narayanapura,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
1,Adugodi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
2,Agaram,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
3,Agrahara Dasarahalli,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
4,Alamo Square,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.027027,0.0,0.0,0.0,0.0,0.013514


In [107]:
df_venues_sorted = sorted_dataframe(df_grouped)
df_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,A Narayanapura,Dessert Shop,Pharmacy,Bus Stop,Pizza Place,Fast Food Restaurant,Yoga Studio,Food & Drink Shop,Film Studio,Financial or Legal Service,...,Fish Market,Flea Market,Flower Shop,Food,Food Stand,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant
1,Adugodi,Bus Station,Café,Athletics & Sports,Playground,Ice Cream Shop,Yoga Studio,Filipino Restaurant,Film Studio,Financial or Legal Service,...,Fish Market,Flea Market,Flower Shop,Food,Food Court,Food & Drink Shop,Farmers Market,Food Stand,Food Truck,Football Stadium
2,Agaram,Bus Station,Burger Joint,Yoga Studio,Food Stand,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,...,Food,Food & Drink Shop,Food Court,Food Truck,Cycle Studio,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop
3,Agrahara Dasarahalli,Indian Restaurant,Bus Station,Bar,Café,Bagel Shop,Vegetarian / Vegan Restaurant,Bakery,Breakfast Spot,Fried Chicken Joint,...,Film Studio,Garden,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Furniture / Home Store,Flower Shop,Food & Drink Shop,French Restaurant
4,Alamo Square,Bar,Park,Ethiopian Restaurant,Café,Seafood Restaurant,Liquor Store,Record Shop,Sushi Restaurant,BBQ Joint,...,Pizza Place,Hotel,Convenience Store,Pet Store,Hunan Restaurant,Comic Shop,Coffee Shop,Yoga Studio,Cocktail Bar,Restaurant


In [176]:
#set number of clusters
kclusters = 5

df_grouped_clustering = df_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans1 = KMeans(n_clusters=kclusters, random_state=0).fit(df_grouped_clustering)

# check cluster labels generated for each row in the dataframe
len(kmeans1.labels_)

283

**Note** : _df_sf is not sorted, while df_grouped was sorted alphabetically on Neighbourhood_

In [177]:
df.shape

(314, 6)

In [178]:
df_labels = df.sort_values(['Neighbourhood'])
x=[]
for i in df_labels['Neighbourhood']:
    if i in df_grouped['Neighbourhood'].values:
        x.append(True)
    else : x.append(False)
df_labels=df_labels[x]
print(df_labels.shape)

(283, 6)


In [179]:
df_labels['Cluster Labels'] = kmeans1.labels_
df_labels.head() 

,Neighbourhood,City,State,Country,Latitude,Longitude,Cluster Labels
55,A Narayanapura,Bangalore,Karnataka,India,12.993194,77.670377,0
146,Adugodi,Bangalore,Karnataka,India,12.942847,77.610416,0
113,Agaram,Bangalore,Karnataka,India,12.945704,77.637886,0
104,Agrahara Dasarahalli,Bangalore,Karnataka,India,12.979063,77.540682,3
198,Alamo Square,San Francisco,California,USA,37.776360,-122.434689,0


In [180]:
df_labels.sort_index(inplace=True)

In [181]:
df_final = pd.concat([df,df_labels],axis=0)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [182]:
df_final.groupby(['Cluster Labels','City']).count()


Country  Latitude  Longitude  Neighbourhood  \
Cluster Labels City                                                         
0.0            Bangalore           71        71         71             71   
               San Francisco      114       114        114            114   
1.0            Bangalore           16        16         16             16   
               San Francisco        1         1          1              1   
2.0            Bangalore            7         7          7              7   
3.0            Bangalore           71        71         71             71   
4.0            Bangalore            2         2          2              2   
               San Francisco        1         1          1              1   

                              State  
Cluster Labels City                  
0.0            Bangalore         71  
               San Francisco    114  
1.0            Bangalore         16  
               San Francisco      1  
2.0            Bangalore          7  
3.0            Bangalore         71  
4.0            Bangalore          2  
               San Francisco      1

In [183]:
address = 'San Francisco, California'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Francisco are {}, {}.'.format(latitude, longitude))

map_city_clusters(df_labels,latitude,longitude)

The geograpical coordinate of San Francisco are 37.7790262, -122.4199061.


In [184]:
address = 'Bengaluru, India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Francisco are {}, {}.'.format(latitude, longitude))

map_city_clusters(df_labels,latitude,longitude)

The geograpical coordinate of San Francisco are 12.9791198, 77.5912997.


In [185]:
df_labels.loc[df_labels['Cluster Labels']==0,df_labels.columns[[0] + list(range(6, df_labels.shape[1]))]]

,Neighbourhood,Cluster Labels
0,Kempegowda Ward,0
3,Yelahanka Satellite Town,0
5,Thanisandra,0
10,Kuvempu Nagar,0
13,Bagalakunte,0
14,T Dasarahalli,0
21,Vishwanath Nagenahalli,0
23,HBR Layout,0
25,Ramamurthy Nagar,0
27,Kammanahalli,0


In [186]:
df_labels.loc[df_labels['Cluster Labels']==1,df_labels.columns[[0] + list(range(6, df_labels.shape[1]))]]

,Neighbourhood,Cluster Labels
1,Chowdeswari Ward,1
6,Byatarayanapura,1
41,Lakshmi Devi Nagar,1
45,Jayachamarajendra Nagar,1
87,Jeevanbhima Nagar,1
88,Jogupalya,1
91,Shivaji Nagar,1
101,Vrisahbhavathi Nagar,1
132,Hampi Nagar,1
133,Bapuji Nagar,1


In [187]:
df_labels.loc[df_labels['Cluster Labels']==2,df_labels.columns[[0] + list(range(6, df_labels.shape[1]))]]

,Neighbourhood,Cluster Labels
47,Muneshwara Nagar,2
50,Vijnanapura,2
69,Rajagopal Nagar,2
72,Kottegepalya,2
124,Marenahalli,2
134,Padarayanapura,2
136,Rayapuram,2


In [188]:
df_labels.loc[df_labels['Cluster Labels']==3,df_labels.columns[[0] + list(range(6, df_labels.shape[1]))]]

,Neighbourhood,Cluster Labels
2,Atturu,3
8,Vidyaranyapura,3
15,Jalahalli,3
17,Radhakrishna Temple Ward,3
18,SanJayanagar,3
19,Ganga Nagar,3
20,Hebbala,3
22,Nagavara,3
24,Horamavu,3
26,Banasavadi,3
